In [ ]:
import numpy as np
import pandas as pd
import os
from openpyxl import load_workbook
import sys
import re
import calendar


# Paths
csv_folder  = r"D:\TRAINING MODEL\data\csv_folder"
logs_folder = r"D:\TRAINING MODEL\logs"

# File path
log_file = os.path.join(logs_folder, "processed_files.log")

In [ ]:
# Final column schema
final_columns = [
    "Month_year", "Consultation_Type", "Case",
    "Under 1 Male", "Under 1 Female",
    "1-4 Male", "1-4 Female",
    "5-9 Male", "5-9 Female",
    "10-14 Male", "10-14 Female",
    "15-18 Male", "15-18 Female",
    "19-24 Male", "19-24 Female",
    "25-29 Male", "25-29 Female",
    "30-34 Male", "30-34 Female",
    "35-39 Male", "35-39 Female",
    "40-44 Male", "40-44 Female",
    "45-49 Male", "45-49 Female",
    "50-54 Male", "50-54 Female",
    "55-59 Male", "55-59 Female",
    "60-64 Male", "60-64 Female",
    "65-69 Male", "65-69 Female",
    "70 Over Male", "70 Over Female"
]



In [ ]:
# --------------------------
# Load already processed files
# --------------------------
processed_files = set()
if os.path.exists(log_file):
    with open(log_file, "r") as f:
        processed_files = set(line.strip() for line in f.readlines())
        
# --------------------------
# Scan and process only NEW CSV files
# --------------------------
for file in os.listdir(csv_folder):
    if file.endswith(".csv") and file not in processed_files:  # ✅ Skip logged files
        file_path = os.path.join(csv_folder, file)
        try:
            # Read CSV
            df = pd.read_csv(file_path)

            # Drop first column (extra index column)
            df = df.drop(df.columns[0], axis=1)

            # Add 2 new columns on the left
            df.insert(0, "Month_year", "")
            df.insert(1, "Consultation_Type", "")

            # If too many columns → trim
            if df.shape[1] > len(final_columns):
                df = df.iloc[:, :len(final_columns)]

            # If too few columns → pad with empty ones
            while df.shape[1] < len(final_columns):
                df[f"Extra_{df.shape[1]}"] = ""

            # Rename columns
            df.columns = final_columns

            # Save cleaned CSV (overwrites original inside csv_folder)
            df.to_csv(file_path, index=False)

            # Append this file to log
            with open(log_file, "a") as f:
                f.write(file + "\n")

            print(f"✅ Processed and logged: {file}")

        except Exception as e:
            print(f"❌ Error processing {file}: {e}")

print("🎯 All new CSV files processed and logged.")

In [ ]:
# Reload processed files
with open(log_file, "r") as f:
    processed_files = set(line.strip() for line in f.readlines())

# Update Month_year in each processed file
for file in processed_files:
    file_path = os.path.join(csv_folder, file)
    try:
        # Read raw text
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

        # Extract month and year
        match = re.search(r"MONTH AND YEAR:\s*([A-Za-z]+)\s+(\d{4})", text)
        month_year_value = ""
        if match:
            month_name = match.group(1).strip().title()
            year = match.group(2).strip()
            try:
                month_num = list(calendar.month_name).index(month_name)
                month_year_value = f"{year} - {month_num}"
            except ValueError:
                pass

        # Update dataframe
        df = pd.read_csv(file_path)
        df["Month_year"] = month_year_value

        # Save back
        df.to_csv(file_path, index=False)
        print(f"Updated {file} with Month_year={month_year_value}")

    except Exception as e:
        print(f"Error updating {file}: {e}")

print("All Month_year values updated.")


In [ ]:
# Open log file
with open(log_file, "a", encoding="utf-8") as log:
    # Loop through all CSV files
    for file in os.listdir(csv_folder):
        if file.endswith(".csv"):
            file_path = os.path.join(csv_folder, file)

            print(f"\n📂 Processing: {file}")

            # Load CSV
            final_df = pd.read_csv(file_path)

            # Keep a mapping and current category
            current_category = None
            found_categories = []

            # Create a new column for Consultation_Type if not present
            if "Consultation_Type" not in final_df.columns:
                final_df.insert(1, "Consultation_Type", "")

            # Iterate row by row
            for i, row in final_df.iterrows():
                for cell in row.dropna().astype(str):
                    if "TOP 10" in cell.upper():
                        # Extract last word (category)
                        last_word = re.sub(r"[^\w]", "", cell.strip().split()[-1])
                        current_category = last_word.capitalize()
                        found_categories.append((i, current_category))
                        print(f"✅ Found: {current_category} at row {i}")
                        break  # stop after first match in this row

                # Fill Consultation_Type with current_category
                if current_category:
                    final_df.at[i, "Consultation_Type"] = current_category

            # Save updated file (overwrite original in csv_folder)
            final_df.to_csv(file_path, index=False)


            print(f"💾 Saved updated file: {file}")
            print("🔍 Unique categories found:", set(cat for _, cat in found_categories))


In [ ]:
# Loop through all CSV files in folder

drop_indexes = []

for file in os.listdir(csv_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(csv_folder, file)
        print(f"\n📂 Cleaning file: {file}")

        # Load CSV
        final_df = pd.read_csv(file_path)

        # Find rows to drop
        drop_indexes = []
        for i, row in final_df.iterrows():
            for cell in row.dropna().astype(str):
                if "PASIG CITY CHILDREN'S HOSPITAL/PASIG CITY COVID-19 REFERRAL CENTER" in cell.upper():
                    drop_indexes.extend(range(i, i + 9))  # this row + 8    below
                    break

        # Find rows containing "TOTAL"
        for i, row in final_df.iterrows():
            for cell in row.dropna().astype(str):
                if "TOTAL" in cell.upper().strip():
                    # Add this row and the next 2 rows
                    drop_indexes.extend([i, i+1, i+2])
                    break  # stop checking other cells in the same row

        # Drop duplicates in case overlaps happen
        drop_indexes = list(set(drop_indexes))

        # Drop rows and reset index
        final_df = final_df.drop(drop_indexes, errors="ignore").reset_index(drop=True)

        # Save back to CSV
        final_df.to_csv(file_path, index=False)

        print(f"✅ Removed {len(drop_indexes)} rows and updated file: {file_path}")